In [4]:
# Measurement imports
import pymeasure
from pymeasure.adapters import PrologixAdapter
from pymeasure.instruments.keithley import Keithley2400
import time

# Scientific package imports
import numpy as np
import pandas as pd
import plotly as py
import plotly.graph_objs as go

In [7]:
# Instrument connection
def connect_to_sourcemeter(com_port, gpib_address):
    ProAdapter = PrologixAdapter(com_port, address = gpib_address, serial_timeout= 0.1)
    return Keithley2400(ProAdapter)

scm = connect_to_sourcemeter('/dev/tty.usbserial-PXFDLVOK', 24)
    
scm.id # Test connection

'KEITHLEY INSTRUMENTS INC.,MODEL 2400,1293432,C30   Mar 17 2006 09:29:29/A02  /K/J'

In [8]:
scm.id

'KEITHLEY INSTRUMENTS INC.,MODEL 2400,1293432,C30   Mar 17 2006 09:29:29/A02  /K/J'

In [10]:
def is_connected(sourcemeter):
    return sourcemeter.id.startswith("KEITHLEY")

is_connected(scm)

True

In [11]:
def setup_current_measurement(sourcemeter, voltage, current_compliance):
    sourcemeter.reset()
    sourcemeter.apply_voltage(20, 0.1)
    sourcemeter.measure_current(nplc=0.1, current=current_compliance, auto_range=False)
    sourcemeter.source_voltage = voltage
    sourcemeter.sample_continuously()

setup_current_measurement(scm, 5, 0.1)

In [16]:
# Create a figure
fig_cur = go.FigureWidget(
    data = [go.Scatter(x=[0],y=[0])],
    layout = go.Layout(
        xaxis=dict(title='Time (s)', gridwidth=1, zeroline=False, ticklen=1, linecolor = 'black',linewidth = 2, mirror = True),
        yaxis=dict(title='Current (A)', linecolor = 'black',linewidth = 2, mirror = True),
        width=500,
        height=500
    )
)
fig_cur

FigureWidget({
    'data': [{'type': 'scatter', 'uid': 'f1920e3a-9753-4de3-8fec-32e5a2783d8b', 'x': [0], 'y': …

In [17]:
t_start = time.time()
exp_length = 50 # seconds
scm.enable_source()

while True:
    cTime = time.time()
    cCurrent = scm.current
    fig_cur.data[0]['x'] += (cTime - t_start,)
    fig_cur.data[0]['y'] += (cCurrent,)
    if cTime - t_start > exp_length:
        break
    
scm.shutdown()